In [1]:
import os
import tqdm
import pandas as pd
import numpy as np

from modules.data import PocketDataset, Dataloader
from modules.Trainer import Pseq2SitesTrainer

In [2]:
config = {
    "paths": {
        "input_dir_embedding": os.path.join("data_embeddings", "pdbbind"),
        "data_path": os.path.join("data_preprocessed", "PDBbind_data.tsv"),
        "save_path": os.path.join("outputs", "pdbbind", "CV"),
        "model_path": os.path.join("outputs", "pdbbind", "model"),
        "result_path": os.path.join("outputs", "pdbbind", "prediction.tsv")
    },
    "train": {
        "epochs": 50,
        "batch_size": 8,
        "dropout": 0.3
    },
    "architectures": {
        "hidden_size": 256,
        "prots_input_dim": 1024,
        "num_layer": 3,
        "hidden_act": "gelu",
        "hidden_size": 256,
        "intermediate_size": 512,
        "num_attention_heads": 8
    },
    "prots": {
        "max_lengths": 1500,
        "max_chains": 13
    }
}

In [3]:
if not os.path.exists(config["paths"]["save_path"]):
    os.mkdir(config["paths"]["save_path"])
    
if not os.path.exists(config["paths"]["model_path"]):
    os.mkdir(config["paths"]["model_path"])

# 1. Load Data

In [4]:
df_seq = pd.read_csv(config["paths"]["data_path"], sep="\t")
df_seq

,PDB,Sequence,BS
0,4x14,VEVLEVKTGVDSITEVECFLTPEMGDPDEHLRGFSKSISISDTFES...,"361,362,364,367,368,369,370,371,372,373,374,50..."
1,4ruu,TRDQNGTWEMESNENFEGYMKALDIDFATRKIAVRLTQTLVIDQDG...,"1,2,3,4,5,7,9,12,14,15,16,17,18,19,20,21,22,23..."
2,5hx8,IVSEKKPATEVDPTHFEKRFLKRIRDLGEGHFGKVELCRYDPEGDN...,"23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,4..."
3,2ymt,SAPIPDLKVFEREGVQLNLSFIRPPENPALLLITITATNFSEGDVT...,"16,18,32,34,47,48,49,50,51,52,53,54,55,56,57,5..."
4,4km2,MVGLIWAQATSGVIGRGGDIPWRLPEDQAHFREITMGHTIVMGRRT...,"2,3,4,5,6,7,8,13,14,15,17,18,19,20,21,22,23,24..."
...,...,...,...
9973,5ew9,QWALEDFEIGRPLGKGKFGNVYLAREKQSKFILALKVLFKAQLEKA...,"10,11,12,13,14,15,16,17,18,19,20,21,22,23,30,3..."
9974,4f7l,DKMDYDFKVKLSSERERVEDLFEYEGCKVGRGTYGHVYKAKRKDGK...,"26,27,28,29,30,31,32,33,34,35,36,37,38,39,49,5..."
9975,4elh,MIVSFMVAMDENRVIGKDNNLPWRLPSELQYVKKTTMGHPLIMGRK...,"3,4,5,6,7,8,9,12,13,14,15,16,17,18,19,20,21,22..."
9976,4o3a,GANKTVVVTTILESPYVMMKKNHEMLEGNERYEGYCVDLAAEIAKH...,"9,10,11,12,13,14,15,35,55,58,59,60,61,62,63,70..."


In [5]:
protein_feats = []
for pdbid in tqdm.tqdm(df_seq["PDB"].values):
    filepath = os.path.join(config["paths"]["input_dir_embedding"], f"{pdbid}.npy")
    protein_feats.append(np.load(filepath))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9978/9978 [00:05<00:00, 1675.55it/s]


# 2. 5-fold CV

In [6]:
class KFold:
    def __init__(self, n_splits, random_state):
        self.n_splits = n_splits
        self.random_state = random_state
        self.rng = np.random.default_rng(seed=self.random_state)
        
    def split(self, X):
        total_records = len(X)
        fold_size = total_records // self.n_splits
        remainder = total_records % self.n_splits
        
        ## index permutated
        permuted_idx = self.rng.permutation(total_records)
        
        ## fold
        folds = []
        i = 0
        while i < total_records:
            if len(folds) < remainder:
                i_next = i + fold_size + 1
            else:
                i_next = i + fold_size
            
            folds.append(permuted_idx[i:i_next])
            i = i_next

        ## generator
        for i in range(self.n_splits):
            test_idx = folds[i]
            train_idx = []
            for j in range(self.n_splits):
                if j != i:
                    train_idx.append(folds[j])
            train_idx = np.concatenate(train_idx)
            yield train_idx, test_idx

In [7]:
kf = KFold(5, 2025)

for idx, (train_idx, val_idx) in enumerate(kf.split(df_seq)):
    ## training data
    train_IDs = df_seq.loc[train_idx, "PDB"].values.tolist()
    train_BS = df_seq.loc[train_idx, "BS"].values.tolist()
    train_seqs = df_seq.loc[train_idx, "Sequence"].values.tolist()
    train_feats = [protein_feats[i] for i in train_idx]
    
    ## validation data
    valid_IDs = df_seq.loc[val_idx, "PDB"].values.tolist()
    valid_BS = df_seq.loc[val_idx, "BS"].values.tolist()
    valid_seqs = df_seq.loc[val_idx, "Sequence"].values.tolist()
    valid_feats = [protein_feats[i] for i in val_idx]
    
    ## data loader
    train_dataset = PocketDataset(train_IDs, train_feats, train_seqs, train_BS)
    train_loader = Dataloader(train_dataset, batch_size=config["train"]["batch_size"], shuffle=True)
    
    valid_dataset = PocketDataset(valid_IDs, valid_feats, valid_seqs, valid_BS)
    valid_loader = Dataloader(valid_dataset, batch_size=config["train"]["batch_size"], shuffle=False)
    
    ## output_directory
    output_dir = os.path.join(config["paths"]["save_path"], f"fold{idx}")
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    
    ## run
    print(f"Fold {idx} is running ...")
    trainer = Pseq2SitesTrainer(config)
    history = trainer.train(train_loader, valid_loader, output_dir)
    
    ## save
    pd.DataFrame(history).to_csv(os.path.join(output_dir, "history.csv"), index=False)

Fold 0 is running ...


  0%|                                                                                                                                                                   | 0/50 [00:00<?, ?it/s]

Epoch: 0 / 50, Train Loss: 0.625, Validation Loss: 0.588


  2%|███                                                                                                                                                     | 1/50 [02:05<1:42:51, 125.95s/it]

Epoch: 1 / 50, Train Loss: 0.597, Validation Loss: 0.568


...

Epoch: 47 / 50, Train Loss: 0.170, Validation Loss: 0.288


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 49/50 [1:43:56<02:06, 126.89s/it]

Epoch: 48 / 50, Train Loss: 0.166, Validation Loss: 0.318


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [1:46:03<00:00, 127.27s/it]

Epoch: 49 / 50, Train Loss: 0.163, Validation Loss: 0.298
